# **CIS 6200 Spring 2024 Homework 1**


Objective:
*   Implement context Chunking
*   Learn about Longformer and how it works


Tasks:

*  Complete the **Context Chunking** section of this notebook by filling out the code. Experiment with different hyperparameters and answer the question at the end of the section in PDF.
  * Question 1: Explain Chunking in a few sentences.
  * Question 2: Why does chunking fail in the later example? Can you give another example on which chunking also fails?
*  Answer the questions about **LongFormer** and complete the second section. Report your results and compare with results from chunking.
  * Question 3: Read the [Longformer paper](https://arxiv.org/abs/2004.05150).
    * What is the Sliding Window Attention and Global Attention for Longformer?
    * How do they capture information in a long context?
  * Question 4: Read about the ["Lost in the Middle"](https://arxiv.org/abs/2307.03172) phenomena of LLMs.
    * What is the phenomena describing? What might be a reason for it to happen?
    * Based on your understanding of Longformer, will it also suffer the same problem? **(Answer in PDF)

**Note: Answers to the task questions need to be submitted in the corresponding PDF submission along with this coding submission on gradescope.**

## Setup

In [34]:
!pip -q install wikipedia

In [35]:
import wikipedia
from transformers import LongformerTokenizer, LongformerForQuestionAnswering, AutoTokenizer, AutoModelForQuestionAnswering
import random
import torch
import torch.nn.functional as F
import numpy as np

In [36]:
wikipedia.page("University of Pennsylvania").summary

"The University of Pennsylvania (Penn or UPenn) is a private Ivy League research university in Philadelphia, Pennsylvania. It is one of nine colonial colleges and was chartered prior to the U.S. Declaration of Independence when Benjamin Franklin, the university's founder and first president, advocated for an educational institution that trained leaders in academia, commerce, and public service. Penn identifies as the fourth oldest institution of higher education in the United States, though this representation is challenged by other universities, as Franklin first convened the board of trustees in 1749, arguably making it the fifth oldest institution of higher education in the U.S.The university has four undergraduate schools and 12 graduate and professional schools. Schools enrolling undergraduates include the College of Arts and Sciences, the School of Engineering and Applied Science, the Wharton School, and the School of Nursing. Among its highly ranked graduate schools are its law 

In [37]:
# A piece of longer information collected by querying wikipedia.page("University of Pennsylvania").content

wiki_info = """
The University of Pennsylvania (Penn or UPenn) is a private Ivy League research university in Philadelphia, Pennsylvania. It is one of nine colonial colleges and was chartered prior to the U.S. Declaration of Independence when Benjamin Franklin, the university's founder and first president, advocated for an educational institution that trained leaders in academia, commerce, and public service. Penn identifies as the fourth oldest institution of higher education in the United States, though this representation is challenged by other universities, as Franklin first convened the board of trustees in 1749, arguably making it the fifth oldest institution of higher education in the U.S.The university has four undergraduate schools and 12 graduate and professional schools. Schools enrolling undergraduates include the College of Arts and Sciences, the School of Engineering and Applied Science, the Wharton School, and the School of Nursing. Among its highly ranked graduate schools are its law school, whose first professor James Wilson participated in writing the first draft of the U.S. Constitution, its medical school, which was the first medical school established in North America, and Wharton, the nation's first collegiate business school. Penn's endowment is $20.7 billion, making it the sixth-wealthiest private academic institution in the nation as of 2022. In 2021, it ranked 4th among American universities in research expenditures according to the National Science Foundation.The University of Pennsylvania's main campus is located in the University City neighborhood of West Philadelphia, and is centered around College Hall. Notable campus landmarks include Houston Hall, the first modern student union, and Franklin Field, the nation's first dual-level college football stadium and the nation's longest-standing NCAA Division I college football stadium in continuous operation. The university's athletics program, the Penn Quakers, fields varsity teams in 33 sports as a member of NCAA Division I's Ivy League conference.
Since its founding, Penn alumni, trustees, and faculty have included 8 signers of the U.S. Declaration of Independence, 7 signers of the Constitution, 3 Presidents of the United States, 3 U.S. Supreme Court justices, 32 U.S. senators, 163 members of the U.S. House of Representatives, 19 U.S. Cabinet Secretaries, 46 governors, 28 State Supreme Court justices, and 9 foreign heads of state. Alumni and faculty include 39 Nobel laureates, 4 Turing Award winners, and a Fields Medalist. Penn has graduated 32 Rhodes Scholars and 21 Marshall Scholars. As of 2022, Penn has the largest number of undergraduate alumni who are billionaires of all colleges and universities (17, counting only Penn's four undergraduate schools). Penn alumni have won (a) 53 Tony Awards, (b) 17 Grammy Awards, (c) 25 Emmy Awards, and (d) 13 Academy Awards. At least 43 different Penn alumni have earned 81 Olympic medals (26 gold), 2 Penn alumni have been NASA astronauts, and 5 Penn alumni have been awarded the Medal of Honor.

Since the Penn Museum was founded in 1887, it has taken part in 400 research projects worldwide. The museum's first project was an excavation of Nippur, a location in current day Iraq.Penn Museum is home to the largest authentic sphinx in North America at about seven feet high, four feet wide, 13 feet long, and 12.9 tons (made of solid red granite).
The sphinx was discovered in 1912 by the British archeologist, Sir William Matthew Flinders Petrie, during an excavation of the ancient Egyptian city of Memphis, Egypt, where the sphinx had guarded a temple to ward off evil. Since Petri's expedition was partially financed by Penn Petrie offered it to Penn, which arranged for it to be moved to museum in 1913. The sphinx was moved in 2019 to a more prominent spot intended to attract visitors.The museum has three gallery floors with artifacts from Egypt, the Middle East, Mesoamerica, Asia, the Mediterranean, Africa and indigenous artifacts of the Americas. Its most famous object is the goat rearing into the branches of a rosette-leafed plant, from the royal tombs of Ur.
The Penn Museum's excavations and collections foster a strong research base for graduate students in the Graduate Group in the Art and Archaeology of the Mediterranean World. Features of the Beaux-Arts building include a rotunda and gardens that include Egyptian papyrus.

Penn's educational innovations include the nation's first medical school in 1765; the first university teaching hospital in 1874; the Wharton School, the world's first collegiate business school, in 1881; the first American student union building, Houston Hall, in 1896; the only school of veterinary medicine in the United States that originated directly from its medical school, in 1884; and the home of ENIAC, the world's first electronic, large-scale, general-purpose digital computer in 1946. Penn is also home to the oldest continuously functioning psychology department in North America and is where the American Medical Association was founded. In 1921, Penn was also the first university to award a PhD in economics to an African American woman, Sadie Tanner Mossell Alexander.
"""

# Context Chunking

**Question 1:** \
Explain Chunking in a few sentences. **(Answer in PDF)**

You can use [LangChain](https://python.langchain.com/docs/modules/data_connection/document_transformers/#evaluate-text-splitters) for a reference.

**Code Implementation**

In [38]:
# TODO: Setup model "distilbert-base-cased-distilled-squad"

checkpoint="distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

In [79]:
def process_with_chunks(text, question, chunk_size=256):

    # TODO: Tokenize question and context
    question_tokens = tokenizer.encode(question)
    context_tokens = tokenizer.encode(text)

    # TODO: Chunk the context according to chuck_size
    chunks = [context_tokens[i:i + chunk_size] for i in range(0, len(context_tokens), chunk_size)]

    # Run the model on each chuck to get outputs
    all_answers = []
    all_scores = []
    for chunk in chunks:

        # Prepare input tokens. Note: [CLS] and [SEP] are BERT tokens, to learn more: https://datascience.stackexchange.com/questions/51522/what-is-the-use-of-sep-in-paper-bert
        input_tokens = ['[CLS]'] + question_tokens + ['[SEP]'] + chunk + ['[SEP]']

        #Prepare input tokens
        input_ids=tokenizer.build_inputs_with_special_tokens(question_tokens, chunk)

        #Convert to PyTorch tensors
        input_ids_tensor=torch.tensor([input_ids])

        outputs=model(input_ids_tensor)
        answer_start_scores=outputs.start_logits
        answer_end_scores=outputs.end_logits

        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1 #Adding 1 because the end index is exclusive

        # Query the model for outputs, extract and decode the answer by selecting the best start / end logits
        answer = tokenizer.decode(input_ids[answer_start:answer_end])
        all_answers.append(answer)
        scores = answer_start_scores[0, answer_start.item()] + answer_end_scores[0, answer_end.item()]
        all_scores.append(scores)

    # Get the most-likely right answer by taking the max of scores
    best_answer = all_answers[torch.argmax(torch.tensor(all_scores))]

    return all_answers, all_scores, best_answer

In [78]:
# Example that success with BERT
text = wikipedia.page("University of Pennsylvania").summary
question = "Where is upenn"

answers, scores, best_answer = process_with_chunks(text, question)
print("All answers: ", answers, "\nAll scores: ", scores, "\nBest answer: ", best_answer)

All answers:  ['Philadelphia, Pennsylvania', '[CLS]', 'Penn'] 
All scores:  [tensor(11.5886, grad_fn=<AddBackward0>), tensor(-0.6043, grad_fn=<AddBackward0>), tensor(2.4190, grad_fn=<AddBackward0>)] 
Best answer:  Philadelphia, Pennsylvania


In [41]:
# Example that fails with BERT
text = wiki_info
question = "Where is Penn Museum"

answers, scores, best_answer = process_with_chunks(text, question)
print("All answers: ", answers, "\nAll scores: ", scores, "\nBest answer: ", best_answer)

All answers:  ['Philadelphia, Pennsylvania', '[CLS]', '[CLS]', '', 'Penn Museum'] 
All scores:  [tensor(7.2992, grad_fn=<AddBackward0>), tensor(6.6112, grad_fn=<AddBackward0>), tensor(-4.1207, grad_fn=<AddBackward0>), tensor(-1.0483, grad_fn=<AddBackward0>), tensor(-6.9324, grad_fn=<AddBackward0>)] 
Best answer:  Philadelphia, Pennsylvania


In [95]:
# Own Example that fails with BERT
text = wiki_info
question = "Where is Memphis located?"#Correct answer is Egypt

answers, scores, best_answer = process_with_chunks(text, question)
print("All answers: ", answers, "\nAll scores: ", scores, "\nBest answer: ", best_answer)

All answers:  ['Philadelphia', 'West Philadelphia', 'Egypt', '[CLS]', 'Penn'] 
All scores:  [tensor(3.8445, grad_fn=<AddBackward0>), tensor(-2.0553, grad_fn=<AddBackward0>), tensor(1.2888, grad_fn=<AddBackward0>), tensor(-2.1990, grad_fn=<AddBackward0>), tensor(-10.9857, grad_fn=<AddBackward0>)] 
Best answer:  Philadelphia


**Question 2:** \
Why does chunking fail in the later example? Can you give another example on which chunking also fails? **(Answer in PDF)**

# Longformer

**Question 3** Read the [Longformer paper](https://arxiv.org/abs/2004.05150).
  * What is the Sliding Window Attention and Global Attention for Longformer?
  * How do they capture information in a long context? **(Answer in PDF)**

**Code Implementation**

In [82]:
# Initialize the models

longformer_tokenizer = AutoTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")
longformer_model = AutoModelForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")

Some weights of the model checkpoint at valhalla/longformer-base-4096-finetuned-squadv1 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [83]:
def process_with_longformer(text, question):
    # Tokenize the input text
    encoding = longformer_tokenizer.encode_plus(question, text, return_tensors="pt")
    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]

    # Get the predictions
    outputs = longformer_model(input_ids, attention_mask=attention_mask)
    start_scores, end_scores = outputs.start_logits, outputs.end_logits

    # Convert predictions into answer
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1
    answer = longformer_tokenizer.decode(input_ids[0, start_index:end_index], skip_special_tokens=True)

    # Print answer
    return(answer)

In [84]:
# Success Example from section 1
text = wikipedia.page("University of Pennsylvania").summary
question = "Where is upenn"


answer = process_with_longformer(text, question)
answer

' Philadelphia, Pennsylvania'

In [85]:
# Failed Example from section 1 -- is it working now?
text = wikipedia.page("University of Pennsylvania").summary
question = "Where is Penn Museum"


answer = process_with_longformer(text, question)
answer

' Philadelphia, Pennsylvania'

In [96]:
# Experiment with your other example from section 1 and verify it works

text = wikipedia.page("University of Pennsylvania").summary
question = "Where is Memphis located?"#Correct answer is Egypt

answer = process_with_longformer(text, question)
answer

' Philadelphia, Pennsylvania'

**Question 4**
Read about the ["Lost in the Middle"](https://arxiv.org/abs/2307.03172) phenomena of LLMs.
* What is the phenomena describing? What might be a reason for it to happen?
* Based on your understanding of Longformer, will it also suffer the same problem? **(Answer in PDF)**
